Загрузим данные.

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from pandas import Series, DataFrame


df = pd.read_excel("NER.xlsx")
df["words"] = df["words"].astype(str)
df[1:12]

,words,Город,Контрагент,ФИО
1,не,О,О,О
2,подтягивается,О,О,О
3,кз,О,О,О
4,в,О,О,О
5,пакет,О,О,О
6,эдо,О,О,О
7,скрин,О,О,О
8,нестандартный,О,О,О
9,зно,О,О,О
10,---,О,О,О


Рассмотрим примеры каждой именованной сущности:

- Контрагент:

In [2]:
df[df["Контрагент"] != "О"][:10]

,words,Город,Контрагент,ФИО
365,ооо,О,н,О
366,абутилон,О,п,О
367,строй,О,п,О
463,индивидуальный,О,н,О
464,агава,О,п,О
465,габов,О,п,О
466,иван,О,п,О
467,михайлович,О,п,О
1033,ип,О,н,О
1034,агапантус,О,п,О


- Фамилия, имя и отчество:

In [3]:
df[df["ФИО"] != "О"][:10]

,words,Город,Контрагент,ФИО
319,прищепова,О,О,н
320,альбина,О,О,п
654,мазитовой,О,О,н
1076,макарова,О,О,н
1847,ильбаев,О,О,н
1848,халит,О,О,п
1849,абдулхабирович,О,О,п
3687,коптяева,О,О,н
3694,афанвсьева,О,О,н
3695,анатолия,О,О,п


- Город:

In [4]:
df[df["Город"] != "О"][:9]

,words,Город,Контрагент,ФИО
377,москва,н,О,О
2355,красноярск,н,О,О
6918,астрахани,н,О,О
6940,самару,н,О,О
10198,канск,н,О,О
12848,нижний,н,О,О
12849,новгород,п,О,О
12852,нижний,н,О,О
12853,новгород,п,О,О


In [5]:
df["sent_number"] = (df["words"] == "---").astype(int).cumsum()
df[8:13]

,words,Город,Контрагент,ФИО,sent_number
8,нестандартный,О,О,О,0
9,зно,О,О,О,0
10,---,О,О,О,1
11,ошибка,О,О,О,1
12,при,О,О,О,1


In [6]:
from pymorphy2 import MorphAnalyzer

lemmatizer = MorphAnalyzer()
to_normal_form = { word : lemmatizer.normal_forms(word) for word in df["words"].unique() }
df["words"] = df["words"].apply(lambda x : to_normal_form[x][0])

In [7]:
df.head()

,words,Город,Контрагент,ФИО,sent_number
0,дд,О,О,О,0
1,не,О,О,О,0
2,подтягиваться,О,О,О,0
3,кз,О,О,О,0
4,в,О,О,О,0


In [8]:
from sklearn.model_selection import train_test_split

df_train, df_dev = train_test_split(df, test_size=0.2)
print("===== полученные размерности =====")
print("df_train.shape:", df_train.shape)
print("df_dev.shape:", df_dev.shape)

===== полученные размерности =====
df_train.shape: (639058, 5)
df_dev.shape: (159765, 5)


## Базовый подход

Базовый подход заключается просто в запоминании тегов для всех слов из тренировочного множества. Более подробно:
- Для каждого уникального слова храним словарь, в котором записываем, сколько раз в тренировочном множестве встречались метки $О$, $н$ и $п$
- Запоминаем самую частотную метку для каждого слова
- Во время тестирования для каждого слова выдаем самую частотную метку

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin
from collections import defaultdict

class BasicModel(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y):
        words = Series(X).unique().tolist()
        self.vocabulary = {word : {"О" : 0, "н" : 0, "п" : 0} for word in words}
        for word, entity in zip(X, y):
            self.vocabulary[word][entity] += 1

        self.memory = {}
        for key, dictionary in self.vocabulary.items():
            self.memory[key] = max(dictionary, key=dictionary.get)
    
    def predict(self, X, y=None):

        return [self.memory.get(x, 'О') for x in X]

In [10]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score

X_train = df_train["words"].values.tolist()
X_dev = df_dev["words"].values.tolist()
for name in ["Контрагент", "Город", "ФИО"]:
    y_train = df_train[name].values.tolist()
    y_dev = df_dev[name].values.tolist()
    basic_model = BasicModel()
    basic_model.fit(X_train, y_train)
    print("=====", name, "=====")
    print("precision_score (начало сущности):", precision_score(basic_model.predict(X_dev), y_dev, average=None)[1])
    print("precision_score (продолжение сущности):", precision_score(basic_model.predict(X_dev), y_dev, average=None)[2])
    print("recall_score (начало сущности):", recall_score(basic_model.predict(X_dev), y_dev, average=None)[1])
    print("recall_score (продолжение сущности):", recall_score(basic_model.predict(X_dev), y_dev, average=None)[2])

===== Контрагент =====
precision_score (начало сущности): 0.4157303370786517
precision_score (продолжение сущности): 0.06299212598425197
recall_score (начало сущности): 0.7254901960784313
recall_score (продолжение сущности): 0.8
===== Город =====
precision_score (начало сущности): 0.8035714285714286
precision_score (продолжение сущности): 0.5675675675675675
recall_score (начало сущности): 0.8587786259541985
recall_score (продолжение сущности): 0.9545454545454546
===== ФИО =====
precision_score (начало сущности): 0.5404290429042904
precision_score (продолжение сущности): 0.845564773452457
recall_score (начало сущности): 0.8768406961178046
recall_score (продолжение сущности): 0.9137931034482759


## Неструктурированные методы

Прежде чем использовать стандартные методы машинного обучения, нам необходимо каждому объекту из выборки (в нашем случае объекты - это слова) сопоставить признаковое описание. Каждому слову будем сопоставлять следующую информацию:
- длину слова
- является ли слово числом
- содержит ли слово символы, отличные от букв

In [11]:
def feature_map(word):
    
    return np.array([len(word), word.isdigit(), word.isalpha()])

In [12]:
X_train = [feature_map(w) for w in df_train["words"].values.tolist()]
X_dev = [feature_map(w) for w in df_dev["words"].values.tolist()]

Теперь мы можем применить стандартные модели. Начем со случайного леса и посмотрим на результат его работы для каждой именованной сущности:

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
for name in ["Контрагент", "Город", "ФИО"]:
    y_train = df_train[name].values.tolist()
    y_dev = df_dev[name].values.tolist()
    model = RandomForestClassifier(𝑐𝑙𝑎𝑠𝑠_𝑤𝑒𝑖𝑔ℎ𝑡 ={"О" : 1, "н" : 1000, "п" : 1000})
    model.fit(X_train, y_train)
    rf_prediction = model.predict(X_dev)
    print("=====", name, "=====")
    print("precision_score (начало сущности):", precision_score(rf_prediction, y_dev, average=None)[1])
    print("precision_score (продолжение сущности):", precision_score(rf_prediction, y_dev, average=None)[2])
    print("recall_score (начало сущности):", recall_score(rf_prediction, y_dev, average=None)[1])
    print("recall_score (продолжение сущности):", recall_score(rf_prediction, y_dev, average=None)[2])

===== Контрагент =====
precision_score (начало сущности): 0.5730337078651685
precision_score (продолжение сущности): 0.7952755905511811
recall_score (начало сущности): 0.0024182076813655763
recall_score (продолжение сущности): 0.0013928152795973246
===== Город =====
precision_score (начало сущности): 0.95
precision_score (продолжение сущности): 0.0
recall_score (начало сущности): 0.003025512119109635
recall_score (продолжение сущности): 0.0
===== ФИО =====
precision_score (начало сущности): 0.4744224422442244
precision_score (продолжение сущности): 0.7402680280791321
recall_score (начало сущности): 0.01468485034222086
recall_score (продолжение сущности): 0.01394381603779255


Теперь используем логистическую регрессию:

In [14]:
for name in ["Контрагент", "Город", "ФИО"]:
    y_train = df_train[name].values.tolist()
    y_dev = df_dev[name].values.tolist()
    model = LogisticRegression(solver='lbfgs',𝑐𝑙𝑎𝑠𝑠_𝑤𝑒𝑖𝑔ℎ𝑡 ={"О" : 1, "н" : 1000, "п" : 1000})
    model.fit(X_train, y_train)
    lr_prediction = model.predict(X_dev)
    print("=====", name, "=====")
    print("precision_score (начало сущности):", precision_score(lr_prediction, y_dev, average=None)[1])
    print("precision_score (продолжение сущности):", precision_score(lr_prediction, y_dev, average=None)[2])
    print("recall_score (начало сущности):", recall_score(lr_prediction, y_dev, average=None)[1])
    print("recall_score (продолжение сущности):", recall_score(lr_prediction, y_dev, average=None)[2])

===== Контрагент =====
precision_score (начало сущности): 0.7191011235955056
precision_score (продолжение сущности): 0.6220472440944882
recall_score (начало сущности): 0.0013076168682576005
recall_score (продолжение сущности): 0.0013458950201884253
===== Город =====
precision_score (начало сущности): 0.9964285714285714
precision_score (продолжение сущности): 0.0
recall_score (начало сущности): 0.0024024386043467777
recall_score (продолжение сущности): 0.0
===== ФИО =====
precision_score (начало сущности): 0.4777227722772277
precision_score (продолжение сущности): 0.7453733248245055
recall_score (начало сущности): 0.01252650253126217
recall_score (продолжение сущности): 0.011857747637079826


# Структурированные методы

Теперь будем использовать модель, учитывающую порядок слов в предложении, так что старый подход, когда мы анализировали данные по словам и разбивали выборку на обучающую и отложенную не зависимо от их расположения в тексте, уже не подходит. Для каждого типа сущности будем использовать следующее представление данных:
- Разобьем все слова на предложения (учитывая поле $sent\_number$)
- Каждое предложение представим списком пар $(word, entity)$

In [16]:
sentences = {}

In [17]:
for name in ["ФИО", "Город", "Контрагент"]:
    get_pair_func = lambda s: [(word, entity) for word, entity in zip(s["words"].values.tolist(),
                                                                      s[name].values.tolist())]
    grouped_words = df.groupby("sent_number").apply(get_pair_func)
    sentences[name] = [sentence for sentence in grouped_words]

In [18]:
sentences["ФИО"][:1]

[[('дд', 'О'),
  ('не', 'О'),
  ('подтягиваться', 'О'),
  ('кз', 'О'),
  ('в', 'О'),
  ('пакет', 'О'),
  ('эдо', 'О'),
  ('скрин', 'О'),
  ('нестандартный', 'О'),
  ('зно', 'О')]]

In [19]:
from sklearn.model_selection import train_test_split

train_sentences = {}
dev_sentences = {}
for name in ["ФИО", "Город", "Контрагент"]:
    train_sentences[name], dev_sentences[name] = train_test_split(sentences[name], test_size=0.2)
len(train_sentences[name]), len(dev_sentences[name])

(18514, 4629)

In [20]:
name = "ФИО"

In [21]:
MAX_LEN = 50
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

words = list(set(df["words"].values))
tags = list(set(df[name].values))
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {"О" : 0, "н" : 1, "п" : 2}

def sentences2X(sentences):
    X = [[word2idx[w[0]] for w in s] for s in sentences]
    X = pad_sequences(maxlen=MAX_LEN, sequences=X, padding="post",value=len(words) - 1)
    
    return X

def sentences2y(sentences):
    y = [[tag2idx[w[1]] for w in s] for s in sentences]
    y = pad_sequences(maxlen=MAX_LEN, sequences=y, padding="post", value=tag2idx['О'])
    y = [to_categorical(i, num_classes=len(tags)) for i in y]
    
    return np.array(y)

Using TensorFlow backend.


In [22]:
from keras.preprocessing.sequence import pad_sequences

X_train = sentences2X(train_sentences[name])
X_dev = sentences2X(dev_sentences[name])
y_train = sentences2y(train_sentences[name])
y_dev = sentences2y(dev_sentences[name])

- $begin\_recall$ - $Recall$ для метки $н$
- $continuous\_recall$ - $Recall$ для метки $п$
- $begin\_precision$ - $Precision$ для метки $н$
- $continuous\_precision$ - $Precision$ для метки $п$

In [23]:
import tensorflow as tf

EPS=1e-10
def begin_recall(y_true, y_pred):
    
    return tf.reduce_sum(y_true[::, ::, 1] * y_pred[::, ::, 1]) / (tf.reduce_sum(y_true[::, ::, 1]) + EPS)
def continuous_recall(y_true, y_pred):
    
    return tf.reduce_sum(y_true[::, ::, 2] * y_pred[::, ::, 2]) / (tf.reduce_sum(y_true[::, ::, 2]) + EPS)
def begin_precision(y_true, y_pred):
    
    return tf.reduce_sum(y_true[::, ::, 1] * y_pred[::, ::, 1]) / (tf.reduce_sum(y_pred[::, ::, 1]) + EPS)
def continuous_precision(y_true, y_pred):
    
    return tf.reduce_sum(y_true[::, ::, 2] * y_pred[::, ::, 2]) / (tf.reduce_sum(y_pred[::, ::, 2]) + EPS)

Построим архитектуру нейронной сети:

In [24]:
import keras.layers as L
from keras.models import Model

layer_input = L.Input(shape=(MAX_LEN,))
layer_emb = L.Embedding(input_dim=len(words), output_dim=MAX_LEN, input_length=MAX_LEN)(layer_input)
layer_drop = L.Dropout(0.1)(layer_emb)
layer_lstm = L.RNN(L.SimpleRNNCell(units=100), return_sequences=True)(layer_drop)
layer_output = L.TimeDistributed(L.Dense(len(tags), activation="softmax"))(layer_lstm)
model = Model(layer_input, layer_output)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 50)            1833650   
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 50)            0         
_________________________________________________________________
rnn_1 (RNN)                  (None, 50, 100)           15100     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 50, 3)             303       
Total params: 1,849,053
Trainable params: 1,849,053
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.compile(optimizer="adam", 
              loss="categorical_crossentropy", 
              metrics=[begin_recall, continuous_recall, begin_precision, continuous_precision])

In [26]:
model.fit(X_train, np.array(y_train), 
          batch_size=16, 
          epochs=5,
          validation_data = (X_dev, np.array(y_dev)),
          verbose=1)

Train on 18514 samples, validate on 4629 samples
Epoch 1/5
18514/18514 [==============================] - 47s 3ms/step - loss: 0.0363 - begin_recall: 0.2701 - continuous_recall: 0.4892 - begin_precision: 0.2375 - continuous_precision: 0.4597 - val_loss: 0.0119 - val_begin_recall: 0.4560 - val_continuous_recall: 0.7221 - val_begin_precision: 0.5257 - val_continuous_precision: 0.7223
Epoch 2/5
18514/18514 [==============================] - 46s 2ms/step - loss: 0.0062 - begin_recall: 0.7674 - continuous_recall: 0.7863 - begin_precision: 0.7390 - continuous_precision: 0.7797 - val_loss: 0.0110 - val_begin_recall: 0.5564 - val_continuous_recall: 0.7376 - val_begin_precision: 0.6110 - val_continuous_precision: 0.7708
Epoch 3/5
18514/18514 [==============================] - 47s 3ms/step - loss: 0.0035 - begin_recall: 0.8759 - continuous_recall: 0.7967 - begin_precision: 0.8659 - continuous_precision: 0.7928 - val_loss: 0.0117 - val_begin_recall: 0.5377 - val_continuous_recall: 0.7063 - val_be

Используем LSTM:

In [51]:
layer_input = L.Input(shape=(MAX_LEN,))
layer_emb = L.Embedding(input_dim=len(words), output_dim=MAX_LEN, input_length=MAX_LEN)(layer_input)
layer_drop = L.Dropout(0.1)(layer_emb)
layer_lstm = L.LSTM(128, return_sequences=True)(layer_drop)
layer_output = L.TimeDistributed(L.Dense(len(tags), activation="softmax"))(layer_lstm)
model = Model(layer_input, layer_output)

In [52]:
model.compile(optimizer="adam", 
              loss="categorical_crossentropy", 
              metrics=[begin_recall, continuous_recall, begin_precision, continuous_precision])

In [53]:
history = model.fit(X_train, np.array(y_train), 
          batch_size=16, 
          epochs=5,
          validation_data = (X_dev, np.array(y_dev)),
          verbose=1)

Train on 18514 samples, validate on 4629 samples
Epoch 1/5
18514/18514 [==============================] - 80s 4ms/step - loss: 0.0391 - begin_recall: 0.2272 - continuous_recall: 0.5082 - begin_precision: 0.1998 - continuous_precision: 0.4768 - val_loss: 0.0114 - val_begin_recall: 0.5404 - val_continuous_recall: 0.7400 - val_begin_precision: 0.4312 - val_continuous_precision: 0.6921
Epoch 2/5
18514/18514 [==============================] - 76s 4ms/step - loss: 0.0063 - begin_recall: 0.7619 - continuous_recall: 0.7674 - begin_precision: 0.7366 - continuous_precision: 0.7511 - val_loss: 0.0120 - val_begin_recall: 0.5122 - val_continuous_recall: 0.7220 - val_begin_precision: 0.5982 - val_continuous_precision: 0.7429
Epoch 3/5
18514/18514 [==============================] - 77s 4ms/step - loss: 0.0037 - begin_recall: 0.8899 - continuous_recall: 0.8171 - begin_precision: 0.8783 - continuous_precision: 0.8111 - val_loss: 0.0138 - val_begin_recall: 0.6856 - val_continuous_recall: 0.8107 - val_be